<style>
    @import url('https://fonts.googleapis.com/css2?family=Montserrat:wght@300&display=swap');

    body, div, h1, h2, h3, h4, h5, h6, p, span, a, li, td, th {
        font-family: 'Montserrat', sans-serif;
        font-weight: 300;
    }
</style>    
<div id='3.3-signalgenerator'></div>


### 3.3. SignalGenerator

#### Descripción
La clase `SignalGenerator` es responsable de generar señales de trading basadas en los datos procesados de criptomonedas. Utiliza indicadores técnicos, Bandas de Bollinger y el Índice de Fuerza Relativa (RSI) para identificar oportunidades de compra y venta.

#### Funcionalidad

1. **Atributos**
   - `data`: Un DataFrame de pandas que contiene los datos procesados de criptomonedas, incluyendo indicadores técnicos como las Bandas de Bollinger y el RSI.

2. **Métodos**

   - `__init__(self, data: pd.DataFrame) -> None`
     - **Descripción**: Inicializa la clase `SignalGenerator` con los datos procesados de criptomonedas.
     - **Parámetros**:
       - `data` (pd.DataFrame): El DataFrame que contiene los datos procesados de criptomonedas.

   - `_validate_data(self, data: pd.DataFrame) -> None`
     - **Descripción**: Valida que los datos contengan las columnas requeridas.
     - **Parámetros**:
       - `data` (pd.DataFrame): El DataFrame que contiene los datos procesados de criptomonedas.
     - **Funcionamiento**:
       1. Verifica que las columnas necesarias ('close', 'upper_band', 'lower_band', 'rsi', 'over_sold', 'over_bought') estén presentes en el DataFrame.
       2. Lanza una excepción si faltan columnas requeridas.

   - `generate_signals(self) -> pd.DataFrame`
     - **Descripción**: Genera señales de compra y venta basadas en las Bandas de Bollinger y el RSI.
     - **Funcionamiento**:
       1. Itera sobre los datos y genera señales de compra cuando el precio cierra por debajo de la banda inferior y el RSI está por debajo del umbral de sobreventa.
       2. Genera señales de venta cuando el precio cierra por encima de la banda superior y el RSI está por encima del umbral de sobrecompra.
       3. Almacena las señales generadas en nuevas columnas del DataFrame ('buy' y 'sell').
       4. Devuelve el DataFrame con las señales generadas.

#### Descripción del Proceso de Generación de Señales con el Código Implementado

A continuación, se detallan las condiciones chequeadas en cada iteración del proceso y cómo se implementan en el código.

<div style="text-align: center;">
    <img src="../images/signal-generator.jpg" alt="Diagrama de Secuencia" style="width: 75%;">
</div>

##### 1. **Validación de Datos**
   - **Descripción:** Antes de generar las señales, el sistema valida que los datos proporcionados contengan todas las columnas requeridas para realizar los cálculos necesarios.
   - **Código:** 
     - La función `_validate_data(data)` verifica que las columnas necesarias, como `close`, `upper_band`, `lower_band`, `rsi`, `over_sold`, y `over_bought`, estén presentes en el `DataFrame` proporcionado.
     - Si alguna columna falta, se lanza un `ValueError`.
     ```python
     def _validate_data(self, data: pd.DataFrame) -> None:
         required_columns = {'close', 'upper_band', 'lower_band', 'rsi', 'over_sold', 'over_bought'}
         missing_columns = required_columns - set(data.columns)

         if missing_columns:
             raise ValueError(f"Data is missing required columns: {missing_columns}")
     ```

##### 2. **Iteración sobre las Filas**
   - **Descripción:** Una vez validados los datos, el sistema itera sobre cada fila para evaluar si corresponde una señal de compra o de venta.
   - **Código:** 
     - En el método `generate_signals()`, se recorre el `DataFrame` utilizando un bucle `for` y el índice de cada fila.
     ```python
     for i in df.index:
     ```

##### 3. **Chequeo de la Señal de Compra**
   - **Descripción:** Se verifica si la fila contiene una señal de compra. Esta señal ocurre cuando:
     - El precio de cierre es menor que la banda inferior de Bollinger.
     - El RSI es inferior al umbral de sobreventa.
     - La posición actual es cero (es decir, no se tiene una posición abierta).
   - **Código:**
     - Si se cumple esta condición, se registra el precio de compra en la lista `buy_price` y la señal de venta se marca como `NaN`.
     ```python
     if df.loc[i, 'close'] < df.loc[i, 'lower_band'] and df.loc[i, 'rsi'] < df.loc[i, 'over_sold'] and position == 0:
         position = 1
         buy_price.append(df['close'][i])
         sell_price.append(np.nan)
     ```

##### 4. **Chequeo de la Señal de Venta**
   - **Descripción:** Si no se ha generado una señal de compra, se verifica si la fila contiene una señal de venta. Esta señal ocurre cuando:
     - El precio de cierre es mayor que la banda superior de Bollinger.
     - El RSI es mayor que el umbral de sobrecompra.
     - La posición actual es 1 (es decir, se tiene una posición de compra abierta).
   - **Código:**
     - Si se cumple esta condición, se registra el precio de venta en la lista `sell_price` y la señal de compra se marca como `NaN`.
     ```python
     elif df.loc[i, 'close'] > df.loc[i, 'upper_band'] and df.loc[i, 'rsi'] > df.loc[i, 'over_bought'] and position == 1:
         position = 0
         sell_price.append(df['close'][i])
         buy_price.append(np.nan)
     ```

##### 5. **Acción por Defecto: Mantener**
   - **Descripción:** Si ninguna de las condiciones anteriores se cumple, se mantiene la acción sin realizar ningún cambio (se marca tanto la señal de compra como la de venta como `NaN`).
   - **Código:**
     ```python
     else:
         buy_price.append(np.nan)
         sell_price.append(np.nan)
     ```

##### 6. **Verificación de Datos Válidos**
   - **Descripción:** Antes de realizar los chequeos de las señales, se valida que los valores relevantes (precio de cierre, bandas de Bollinger y RSI) no estén vacíos (`NaN`).
   - **Código:**
     - Si algún valor está ausente, se lanza un `ValueError` indicando la fila con datos inválidos.
     ```python
     if not pd.isna(df.loc[i, 'close']) and \
        not pd.isna(df.loc[i, 'lower_band']) and \
        not pd.isna(df.loc[i, 'rsi']) and \
        not pd.isna(df.loc[i, 'upper_band']):
     ```

##### 7. **Devolver el DataFrame Actualizado**
   - **Descripción:** Una vez procesadas todas las filas, el `DataFrame` se actualiza con las señales de compra y venta correspondientes y se devuelve.
   - **Código:**
     - Las listas `buy_price` y `sell_price` se asignan como nuevas columnas en el `DataFrame`.
     - Finalmente, el `DataFrame` actualizado se retorna.
     ```python
     df['buy'] = buy_price
     df['sell'] = sell_price

     return df
     ```

##### 8. **Manejo de Errores**
   - **Descripción:** Si ocurre un error durante el procesamiento (por ejemplo, datos faltantes o un error inesperado), se captura la excepción y se lanza un `RuntimeError` con un mensaje adecuado.
   - **Código:**
     ```python
     except Exception as e:
         raise RuntimeError(f"Failed to generate signals: {e}")
     ```

Este código implementa un proceso robusto para generar señales de compra y venta basadas en criterios específicos de las Bandas de Bollinger y el RSI, y maneja adecuadamente los casos de error y validación de datos.


#### Ejemplo de Uso

```python
import pandas as pd
from crypto_analysis.signal_generator import SignalGenerator

# Supongamos que 'data' es un DataFrame con los datos procesados de criptomonedas
data = pd.read_csv('processed_crypto_data.csv')

# Crear una instancia de SignalGenerator
signal_generator = SignalGenerator(data)

# Generar señales de compra y venta
signals = signal_generator.generate_signals()

# Mostrar las señales generadas
print(signals[['close', 'buy', 'sell']])
```


[Ver código fuente en GitHub](https://github.com/infante-manu/crypto_analysis/blob/main/crypto_analysis/signal_generator.py)